# Google Mobility Data
# NYT COVID Cases & Death Data


In [1]:
# Dependencies and Setup
import json
import os
import pandas as pd
import urllib.request
import requests
from config import db_pwd, db_user
from sqlalchemy import create_engine

## Store Google CSV into DataFrame

In [2]:
csv_file = "Resources/google_mob_US.csv"
google_data_df = pd.read_csv(csv_file)
google_data_df.head()

,Unnamed: 0,State,date,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
0,0,Alabama,2020-02-15,5.0,2.0,39.0,7.0,2.0,-1.0
1,1,Alabama,2020-02-16,0.0,-2.0,-7.0,3.0,-1.0,1.0
2,2,Alabama,2020-02-17,3.0,0.0,17.0,7.0,-17.0,4.0
3,3,Alabama,2020-02-18,-4.0,-3.0,-11.0,-1.0,1.0,2.0
4,4,Alabama,2020-02-19,4.0,1.0,6.0,4.0,1.0,0.0


### Create a new dataframe with select columns

In [3]:
# Create a dataframe with only the columns you need from the original dataframe:
google_df = google_data_df[["State", "date", "retail_and_recreation",
                            "grocery_and_pharmacy", "parks"]]
google_df

,State,date,retail_and_recreation,grocery_and_pharmacy,parks
0,Alabama,2020-02-15,5.0,2.0,39.0
1,Alabama,2020-02-16,0.0,-2.0,-7.0
2,Alabama,2020-02-17,3.0,0.0,17.0
3,Alabama,2020-02-18,-4.0,-3.0,-11.0
4,Alabama,2020-02-19,4.0,1.0,6.0
...,...,...,...,...,...
456457,Wyoming,2020-07-29,NaN,NaN,NaN
456458,Wyoming,2020-07-30,NaN,NaN,NaN
456459,Wyoming,2020-07-31,NaN,NaN,NaN
456460,Wyoming,2020-08-03,NaN,NaN,NaN


In [4]:
google_us = google_df.rename(columns = {"State":'states', "date":'dates', "retail_and_recreation":'retail_recreation',
                            "grocery_and_pharmacy":'grocery_pharmacy', "parks":'parks'})
google_us

,states,dates,retail_recreation,grocery_pharmacy,parks
0,Alabama,2020-02-15,5.0,2.0,39.0
1,Alabama,2020-02-16,0.0,-2.0,-7.0
2,Alabama,2020-02-17,3.0,0.0,17.0
3,Alabama,2020-02-18,-4.0,-3.0,-11.0
4,Alabama,2020-02-19,4.0,1.0,6.0
...,...,...,...,...,...
456457,Wyoming,2020-07-29,NaN,NaN,NaN
456458,Wyoming,2020-07-30,NaN,NaN,NaN
456459,Wyoming,2020-07-31,NaN,NaN,NaN
456460,Wyoming,2020-08-03,NaN,NaN,NaN


## Store NYT COVID cases and deaths CSV into DataFrame

In [5]:
csv_file = "Resources/COVID-states.csv"
covid_data_df = pd.read_csv(csv_file)
covid_data_df.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [6]:
covid_us = covid_data_df.rename(columns = {"state":'states', "fips": 'fips', "date":'dates', "cases":'cases',
                            "deaths":'deaths'})
covid_us.head()

,dates,states,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


### Connect to local database

In [7]:
rds_connection_string = f"{db_user}:{{db_pwd}}@localhost:5432/mobility_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [8]:
engine.table_names()

['covid_data', 'google_data']

### Use pandas to load csv converted DataFrame into database

In [9]:
google_us.to_sql(name='google_data', con=engine, if_exists='append', index=False)

In [10]:
covid_us.to_sql(name='covid_data', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the tables

In [11]:
pd.read_sql_query('select * from google_data', con=engine).head(10)

,states,retail_recreation,grocery_pharmacy,parks,dates
0,Alabama,5.0,2.0,39.0,2020-02-15
1,Alabama,0.0,-2.0,-7.0,2020-02-16
2,Alabama,3.0,0.0,17.0,2020-02-17
3,Alabama,-4.0,-3.0,-11.0,2020-02-18
4,Alabama,4.0,1.0,6.0,2020-02-19
5,Alabama,-7.0,-6.0,-25.0,2020-02-20
6,Alabama,3.0,0.0,12.0,2020-02-21
7,Alabama,5.0,4.0,46.0,2020-02-22
8,Alabama,3.0,2.0,24.0,2020-02-23
9,Alabama,-2.0,-3.0,-13.0,2020-02-24


In [12]:
pd.read_sql_query('select * from covid_data', con=engine).head(10)

,dates,states,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
5,2020-01-25,California,6,1,0
6,2020-01-25,Illinois,17,1,0
7,2020-01-25,Washington,53,1,0
8,2020-01-26,Arizona,4,1,0
9,2020-01-26,California,6,2,0
